# Quantum control

In [ ]:
# file: seeq/control.py
import numpy as np
from seeq.evolution import evolve
from numbers import Number
import scipy.optimize

## Problem statement

We are going to use the parameterized pulse approach to quantum control. Our model assumes a Hamiltonian of the form
$$H = H_0 + g(t;x) H_1$$
where $g(t)$ is a time-dependent control that will be eventually expanded in a basis of functions
$$g(t;x) = \sum_n x_n f_n(t),$$
or which may have some other more complicated dependency, such as
$$g(t;x) = x_0 \cos(x_1 t + x_2).$$
We will denote this fact by writing $H(t;x).$

Our goal is to control $N$ states, that must suffer a unitary operation $U_g$ after a time $T.$ If the unitary evolution with $H(t;x)$ is given by $U(t),$ we will have
$$U(T) |\psi_n\rangle \simeq U |\psi_n\rangle,\;n=1,\ldots,N.$$
The unitary operator satisfies the Schrödinger equation
$$i \frac{d}{dt} U(t;x) = H(t;x) U(t;x),$$
with $U(0) = 1.$

We measure the quality of our control by studying the "fidelity"
$$F = \frac{1}{N}\mathrm{Re}\left(\sum_n \langle\psi_n | U_g^\dagger U(T)|\psi_n\rangle\right).$$
When the protocol is exact, $F\simeq 1.$ When the unitary is not exactly the one we want, or even when the phases are different, this quantity decreases towards 0.

## State and fidelity gradient

The optimization of our cost function requires us to determine how the states respond to a change in the control. In other words $x \to x + \epsilon$ will lead to a modified univary evolution operator $U(t;x+\epsilon),$ as explained in the appendices of [O. Romero-Isart and J. J. García-Ripoll
Phys. Rev. A 76, 052304 (2007)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.76.052304). To first order in perturbation theory, this unitary operator will satisfy
$$i \frac{d}{dt}U(t;x+\epsilon) = \left[H(t) + \epsilon \frac{\partial H}{\partial x}\right] U(t;x+\epsilon).$$

We introduce an interaction picture to compute the deviation $W$ induced by the perturbation
$$U(t;x+\epsilon) = U(t;x) W(t;x).$$
The new operator satisfies
$$i\frac{d}{dt}W(t;x) = \epsilon U(t;x)^\dagger \frac{\partial H}{\partial x}(t;x) U(t;x) W(t;x).$$

Keeping terms to $\mathcal{O}(\epsilon^2),$
$$W(t;x) = 1 - i \epsilon \int_{t_0}^t \mathrm{d}\tau\, U(\tau;x)^\dagger \frac{\partial H}{\partial x}(\tau;x) U(\tau;x) + \mathcal{O}(\epsilon^2).$$
Or in the original operator, 
$$U(t;x+\epsilon) = U(t;x) - i \epsilon U(t;x) \int_{t_0}^t \mathrm{d}\tau\, U(\tau;x)^\dagger \frac{\partial H}{\partial x}(\tau;x) U(\tau;x) + \mathcal{O}(\epsilon^2).$$

This implies a formula for the derivative of the unitary operator
$$\frac{\partial}{\partial x} U(t;x) = - i \epsilon U(t;x) \int_{t_0}^t \mathrm{d}\tau\, U(\tau;x)^\dagger \frac{\partial H}{\partial x}(\tau;x) U(\tau;x) + \mathcal{O}(\epsilon^2).$$

And from here it follows a formula for the gradient of the cost function
$$\frac{\partial}{\partial x}F = \frac{1}{N}\mathrm{Re}\sum_n \int_0^T \left\langle U_g^\dagger U(T;x) U(\tau;x)^\dagger \frac{\partial H}{\partial x}(\tau;x) U(\tau;x) \right\rangle_{\psi_n}\mathrm{d}\tau.$$
Note that all formulas extend trivially for any dimensionality of the variable $x.$

## Algorithm

In order to compute $\nabla F$ for all variables $x_i,$ we are going to proceed as follows.

1. Compute the backwards evolved states $|\xi_n(0)\rangle = U(T;x)^\dagger U_g |\psi_n\rangle.$

2. Solve the system of equations
\begin{eqnarray}
i\frac{d}{dt}|\psi_n(t;x)\rangle &=& H(t;x)|\psi_n(t;x)\rangle,\\
i\frac{d}{dt}|\xi_n(t;x)\rangle &=& H(t;x)|\xi_n(t;x)\rangle,\\
\frac{d}{dt}f_{n,i} &=& \frac{1}{N}\mathrm{Im}\langle \xi_n(t;x)|\frac{\partial H}{\partial x_i}(t;x)|\psi_n(t;x)\rangle
\end{eqnarray}

3. Estimate the gradient $$\frac{\partial F}{\partial x_i} = \sum_{n,i} f_{n,i}(T).$$

The code below implements this version of parameterized control. In the simplest incantation, it requires a function `H(t,x,ψ)` that returns the product $H(t;x)\psi.$ It also requires the target gate `Ug`, the states that are to be controlled as the columns of `ψ0` and a vector of times `T` used to solve the evolution. In this simple form, the algorithm does not compute $\nabla F,$ and uses Scipy's algoritms for [global optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html) to find the optimal control.

A more refined version of the algorithm is used when you pass the optional argument `dH(t,x,ψ)`, which is a function that returns a list of matrices, `[dH1, dH2,...]` with `dHn` being the result of computing $\frac{\partial H}{\partial x_n}(t;x) \psi.$ When `dH` is passed, the function uses the algorithm above to estimate $\nabla F$ and use this to speed up Scipy's optimization algorithms.

The code from `parametric_control()` relies on the function `evolve()` from [SeeQ](Time evolution.ipynb). It defaults to the Chebyshev algorithm, which provides very accurate estimates of the exponential. However, even if this algorithm is accurate, the precision of the simulation depends on the time steps of the evolution. This information is provided, either by supplying a vector of times `T=[t0,t1,...,tN]`, or making `T` a real number and supplying the number of `steps`. You may check convergence of the control and simulations by doubling the number of steps and inspecting whether the results changed.

In [ ]:
# file: seeq/control.py

def parametric_control(x0, H, ψ0, Ug, T, dH=None, check_gradient=False,
                       steps=100, tol=1e-10, method='chebyshev',
                       debug=False, optimizer='BFGS', **kwdargs):
    """Solve the quantum control problem for a Hamiltonian H acting on
    a basis of states ψ.
    
    Arguments:
    ----------
    H      - Callable object H(t,x,ψ) that applies H(t,x) on ψ
    ψ0     - N x d object with N wavefunctions
    Ug     - Desired quantum operation
    T      - Either a time or a vector of times
    steps  - # time steps in the algorithm (if T is a number)
    tol    - Optimization tolerance
    method - Solution method for the time evolution
    debug  - Return cost functions
    optimizer - Method for scipy.optimize.minimize
    
    Output:
    -------
    x      - Optimal control
    F      - Fidelity"""

    if isinstance(T, Number):
        times = np.linspace(0, T, steps)
    else:
        times = np.array(T)
        steps = len(T)
    ξT = Ug @ ψ0
    
    def bare_cost(x, verbose=False):
        for t, ψ in evolve(ψ0, lambda t,ψ: H(t,x,ψ), times, method=method):
            ψT = ψ
        return -np.vdot(ξT, ψT).real

    def cost_and_gradient(x, verbose=False):
        Hx = lambda t, ψ: H(t, x, ψ)
        ξ = list(ξt for t, ξt in evolve(ξT, Hx, times[-1::-1], method=method))
        ξ.reverse()
        f = np.zeros((len(x), len(times)))
        for (i, ((t, ψt), ξt)) in enumerate(zip(evolve(ψ0, Hx, times, method=method), ξ)):
            ψT = ψt
            f[:,i] = np.array([np.vdot(ξt, dHi) for dHi in dH(t, x, ψt)]).imag
        dFdx = np.array([-scipy.integrate.simps(fx, x=times) for fx in f])
        return -np.vdot(ξT, ψT).real, dFdx
    
    if dH is None:
        r = scipy.optimize.minimize(bare_cost, x0, tol=tol, method=optimizer)
        fn = bare_cost
    else:
        if check_gradient:
            _, dFdx = cost_and_gradient(x0)
            dFdx2 = scipy.optimize.approx_fprime(x0, bare_cost, 1e-6)
            err = np.max(np.abs(dFdx2 - dFdx))
            print(f'max gradient error:   {err}')
            print(f'Finite diff gradient: {dFdx2}')
            print(f'Our estimate:         {dFdx}')
        r = scipy.optimize.minimize(cost_and_gradient, x0, tol=tol, jac=True, method=optimizer)
        fn = cost_and_gradient
    if debug:
        return r, fn
    else:
        return r

## Example

We now provide some examples of application of this algorithm with and without gradients.

### a) Qubit flip

This is a simple control. We have a Hamiltonian
$$H = \Omega \sigma_y$$
and we want to implement a spin flip
$$U = \exp(-i (\pi/2) \sigma_y) = -i \sigma_y = \left(\begin{matrix} 0 & -1 \\ 1 & 0 \end{matrix}\right).$$
The solution to achieve this in time $T$ is to use $\Omega = \pi / 2T.$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def test_qubit_flip():
    # Operators that we use in the Hamiltonian
    σz = np.diag([1., -1.])
    σy = np.array([[0., -1.j],[1.j, 0.]])
    # Desired gate
    Ug = -1j * σy
    # Desired time to execute the gate
    T = 1.0
    # Some initial guess of the control
    x0 = [1.0]
    # We want to control all states
    ψ0 = np.eye(2)
    # A function that implements the control
    H = lambda t, x, ψ: x * (σy @ ψ)
    # Execute
    r = parametric_control([1.0], H, ψ0, Ug, T=1.0, method='eig')
    # Plot the ouptut
    sz = np.array([[t, obs[0,0], obs[0,1]]
                   for t, obs in evolve(ψ0, r.x * σy, np.linspace(0, T, 31),
                                        observables=[σz], method='eig')])
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,4))
    ax1.plot(sz[:,0], sz[:,1], '-', label='$\\uparrow$')
    ax1.plot(sz[:,0], sz[:,2], '--', label='$\\downarrow$')
    ax2.plot(sz[:,0], sz[:,0]*0+r.x, '--', label='control')
    ax1.legend()
    ax1.set_xlabel('$t$')
    ax1.set_ylabel('$\\langle\\sigma_z\\rangle$')
    ax2.set_xlabel('$t$')
    ax2.set_ylabel('$x(t)$')

test_qubit_flip()

To illustrate the role of derivatives, we introduce an extra degree of difficulty, by incorporating a time-dependent control with a constant amplitude `x`, which we optimize
$$H(t,x) = x \cos(t) \sigma_y.$$
In this example, we use the estimate of the derivative of $H$ to speed up the optimization
$$\frac{\partial H}{\partial x} = \cos(t)\sigma_y.$$
This derivative is created in the function `dH(t,x,ψ)`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def test_qubit_flip():
    # Operators that we use in the Hamiltonian
    π = np.pi
    σz = np.diag([1., -1.])
    σy = np.array([[0., -1.j],[1.j, 0.]])
    # Desired gate
    Ug = -1j * σy
    # Desired time to execute the gate
    T = 1.0
    # Some initial guess of the control
    x0 = [1.0]
    # We want to control all states
    ψ0 = np.eye(2)
    # A function that implements the control
    H = lambda t, x, ψ: x[0] * np.cos(t) * (σy @ ψ)
    dH = lambda t, x, ψ: [np.cos(t) * (σy @ ψ)]
    # Execute
    r = parametric_control([1.0], H, ψ0, Ug, T=1.0, dH=dH, check_gradient=True, method='eig')
    xopt = r.x
    # Plot the output
    sz = np.array([[t, obs[0,0], obs[0,1]]
                   for t, obs in evolve(ψ0, lambda t, ψ: H(t, xopt, ψ), np.linspace(0, T, 31),
                                        observables=[σz], method='eig')])
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,4))
    t = sz[:,0]
    ax1.plot(t, sz[:,1], '-', label='$\\uparrow$')
    ax1.plot(t, sz[:,2], '--', label='$\\downarrow$')
    ax2.plot(t, xopt*np.cos(t), '--', label='control')
    ax1.legend()
    ax1.set_xlabel('$t$')
    ax1.set_ylabel('$\\langle\\sigma_z\\rangle$')
    ax2.set_xlabel('$t$')
    ax2.set_ylabel('$x(t)$')

test_qubit_flip()

## b) Qubit drive

We want to excite a qubit that has a fixed gap $\Delta=1$ using a model of the form
$$H(t) = \frac{\Delta}{2}\sigma^z + \epsilon \cos(x t) \sigma^y.$$
If we apply an interaction picture,
$$\psi(t) = e^{-i\Delta \sigma^z t /2} \xi(t) = U_0(t)\xi(t),$$
the new state evolves according to the Schrödinger equation
$$i \partial_t \xi = U_0(t)^\dagger H(t) U_0(t) \xi(t).$$
In other words
$$i \partial_t \xi = \frac{\epsilon}{2} (e^{i x t} + e^{-i x t})(-i\sigma^+ e^{i\Delta t} + i\sigma^- e^{-i\Delta t}).$$
If we apply a rotating wave approximation whereby $x\simeq \Delta$
$$i \partial_t \xi \simeq \frac{\epsilon}{2} (-i\sigma^+  + i\sigma^-) = \frac{\epsilon}{2} \sigma^y,$$
which achieves a total rotation in a time $T=\pi/\epsilon.$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math

def test_qubit_drive():
    # Operators that we use in the Hamiltonian
    π = np.pi
    σz = np.diag([1., -1.])
    σx = np.array([[0, 1.0],[1.0, 0.]])
    σy = np.array([[0., -1.j],[1.j, 0.]])
    # Desired time to execute the gate
    ϵ = 0.15
    T = π/ϵ
    steps = 300
    times = np.linspace(0, T, steps)
    # Desired gate
    Ug = -1j * σy @ scipy.linalg.expm(-1j * T * σz)
    # Some initial guess of the control
    x0 = [1.13]
    # We want to control all states
    ψ0 = np.eye(2)
    # A function that implements the control
    H = lambda t, x, ψ: 0.5*(σz @ ψ) + ϵ * math.cos(x[0] * t) * (σy @ ψ)
    dH = lambda t, x, ψ: [-ϵ * t * math.sin(x[0] * t) * (σy @ ψ)]
    # Execute
    r, cost = parametric_control(x0, H, ψ0, Ug, dH=dH, T=times, debug=True, method='eig')
    xopt = r.x
    # Plot the output
    sz = np.array([[obs[0,0], obs[0,1]]
                   for _, obs in evolve(ψ0, lambda t, ψ: H(t, xopt, ψ), times, observables=[σz], method='eig')])
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,4))
    ax1.plot(times, sz[:,0], '-', label='$\\uparrow$')
    ax1.plot(times, sz[:,1], '--', label='$\\downarrow$')
    ax1.legend()
    ax1.set_xlabel('$t$')
    ax1.set_ylabel('$\\langle\\sigma_z\\rangle$')
    xrange = np.linspace(0, 2, 41)
    costs = np.array([cost([x])[0] for x in xrange])
    ax2.plot(xrange, costs)
    ax2.plot([xopt, xopt], [np.min(costs), np.max(costs)], 'r-.')
    ax2.plot(xrange, r.fun + 0*xrange, 'r-.')
    ax2.set_xlabel('$x/\\pi$')
    ax2.set_ylabel('cost')

test_qubit_drive()

## Tests

The following code is only required for the automated testing of the library.

In [ ]:
# file: seeq/test/test_parametric_control.py

from seeq.control import *

In [ ]:
# file: seeq/test/test_parametric_control.py

import unittest

class TestQControl(unittest.TestCase):
    π = np.pi
    σz = np.array([[1., 0.],[0., -1.]])
    σx = np.array([[0., 1.],[1., 0.]])
    σy = np.array([[0., -1.j],[1.j, 0.]])
    ψ0 = np.eye(2)

    def test_nothing(self):
        """For a qubit to remain the same, we do nothing."""
        Ug = np.eye(2)
        H = lambda t, x, ψ: x * (self.σx @ ψ)
        r = parametric_control([1.0], H, self.ψ0, Ug, T=1.0, tol=1e-8, method='expm')
        self.assertEqual(len(r.x), 1)
        self.assertAlmostEqual(r.x[0], 0.0, delta=1e-7)

    def test_nothing2(self):
        """For a qubit to remain the same, we cancel the frequency."""
        Ug = np.eye(2)
        H = lambda t, x, ψ: x[0] * (self.σx @ ψ) + (1.0 - x[1]) * (self.σz @ ψ)
        r = parametric_control([1.0, 0.1], H, self.ψ0, Ug, T=1.0, tol=1e-8, method='expm')
        self.assertEqual(len(r.x), 2)
        self.assertAlmostEqual(r.x[0], 0.0, delta=1e-7)
        self.assertAlmostEqual(r.x[1], 1.0, delta=1e-7)

    def test_qubit_flip(self):
        """Construct a π/2 pulse."""
        Ug = -1j*self.σy
        H = lambda t, x, ψ: (x * self.σy) @ ψ
        r = parametric_control([1.0], H, self.ψ0, Ug, T=1.0, tol=1e-9, method='expm')
        self.assertEqual(len(r.x), 1)
        self.assertAlmostEqual(r.x[0], self.π/2., delta=1e-7)

    def test_nothing_derivative(self):
        """For a qubit to remain the same, we do nothing (with gradients)."""
        Ug = np.eye(2)
        H = lambda t, x, ψ: x * (self.σx @ ψ)
        dH = lambda t, x, ψ: [self.σx @ ψ]
        r = parametric_control([1.0], H, self.ψ0, Ug, T=1.0, dH=dH, tol=1e-8, method='expm')
        self.assertEqual(len(r.x), 1)
        self.assertAlmostEqual(r.x[0], 0.0, delta=1e-7)

    def test_qubit_flip_derivative(self):
        """Construct a π/2 pulse (with gradients)."""
        Ug = -1j*self.σy
        H = lambda t, x, ψ: (x * self.σy) @ ψ
        dH = lambda t, x, ψ: [self.σy @ ψ]
        r = parametric_control([1.0], H, self.ψ0, Ug, T=1.0, dH=dH, tol=1e-9, method='expm')
        self.assertEqual(len(r.x), 1)
        self.assertAlmostEqual(r.x[0], self.π/2., delta=1e-7)

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestQControl'])
unittest.TextTestRunner(verbosity=2).run(suite1);